In [2]:
from ogb import nodeproppred #import DglNodePropPredDataset

dataset = nodeproppred.NodePropPredDataset(name = 'ogbn-mag')

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph, label = dataset[0] # graph: dgl graph object, label: torch tensor of shape (num_nodes, num_tasks)

In [3]:
import numpy as np
label_idx = np.where(label['paper']<2)
label_idx

(array([   130,    136,    153, ..., 736333, 736364, 736388]),
 array([0, 0, 0, ..., 0, 0, 0]))

In [28]:
a,b = dataset[0]
b['paper']

array([[246],
       [131],
       [189],
       ...,
       [266],
       [289],
       [  1]])

In [6]:
new_train_idx = train_idx['paper'][np.where(np.isin(train_idx['paper'],label_idx[0])==True)] #
new_valid_idx = valid_idx['paper'][np.where(np.isin(valid_idx['paper'],label_idx[0])==True)]
new_test_idx = test_idx['paper'][np.where(np.isin(test_idx['paper'],label_idx[0])==True)]

new_idx = np.concatenate((new_train_idx,new_valid_idx),axis=0)
new_idx = np.concatenate((new_idx,new_test_idx),axis=0)

In [7]:
link1_n1 = graph['edge_index_dict'][('author','affiliated_with','institution')][0]
link1_n2 = graph['edge_index_dict'][('author','affiliated_with','institution')][1]

link2_n1 = graph['edge_index_dict'][('author','writes','paper')][0]
link2_n2 = graph['edge_index_dict'][('author','writes','paper')][1]

link3_n1 = graph['edge_index_dict'][('paper','cites','paper')][0]
link3_n2 = graph['edge_index_dict'][('paper','cites','paper')][1]

link4_n1 = graph['edge_index_dict'][('paper','has_topic','field_of_study')][0]
link4_n2 = graph['edge_index_dict'][('paper','has_topic','field_of_study')][1]

### 挑选子图link

In [8]:
l3n1_idx = np.where(np.isin(link3_n1,new_idx)==True)
l3n2_idx = np.where(np.isin(link3_n2,new_idx)==True)
new_l3n1_idx = np.where(np.isin(l3n1_idx[0],l3n2_idx[0])==True)
new_l3n1 = link3_n1[l3n1_idx][new_l3n1_idx].tolist()
new_l3n2 = link3_n2[l3n1_idx][new_l3n1_idx].tolist()

l4n1_idx = np.where(np.isin(link4_n1,new_idx)==True)
new_l4n1 = link4_n1[l4n1_idx].tolist()
new_l4n2 = link4_n2[l4n1_idx].tolist()

l2n2_idx = np.where(np.isin(link2_n2,new_idx)==True)
new_l2n1 = link2_n1[l2n2_idx].tolist()
new_l2n2 = link2_n2[l2n2_idx].tolist()

l1n1_idx = np.where(np.isin(link1_n1,link2_n1[l2n2_idx])==True)
new_l1n1 = link1_n1[l1n1_idx].tolist()
new_l1n2 = link1_n1[l1n1_idx].tolist()

In [9]:
# link_n1 = new_l1n1+new_l2n1+new_l3n1+new_l4n1
# link_n2 = new_l1n2+new_l2n2+new_l3n2+new_l4n2
# link_ltype = [0]*len(new_l1n1)+[1]*len(new_l2n1)+[2]*len(new_l3n1)+[3]*len(new_l4n1)
# feats = ['']*len(new_l1n1)+['']*len(new_l2n1)+['']*len(new_l3n1)+['']*len(new_l4n1)

# import pandas as pd
# links = pd.DataFrame()
# links['node_id1']=link_n1
# links['node_id2']=link_n2
# links['link_type']=link_ltype
# links['feats']=feats

import pandas as pd
link1 = pd.DataFrame()
link1['node_id1']=new_l1n1
link1['node_id2']=new_l1n2
link1['link_type']=[0]*len(new_l1n1)
link1['feats']=['']*len(new_l1n1)

link2 = pd.DataFrame()
link2['node_id1']=new_l2n1
link2['node_id2']=new_l2n2
link2['link_type']=[1]*len(new_l2n1)
link2['feats']=['']*len(new_l2n1)

link3 = pd.DataFrame()
link3['node_id1']=new_l3n1
link3['node_id2']=new_l3n2
link3['link_type']=[2]*len(new_l3n1)
link3['feats']=['']*len(new_l3n1)

link4 = pd.DataFrame()
link4['node_id1']=new_l4n1
link4['node_id2']=new_l4n2
link4['link_type']=[3]*len(new_l4n1)
link4['feats']=['']*len(new_l4n1)

### 挑选子图node

In [10]:
paper = set()
paper.update(tuple(new_l2n2))
paper.update(tuple(new_l3n1))
paper.update(tuple(new_l3n2))
paper.update(tuple(new_l4n1))
paper = list(sorted(paper))

author = set()
author.update(tuple(new_l1n1))
author.update(tuple(new_l2n1))
author = list(sorted(author))

institution = set()
institution.update(tuple(new_l1n2))
institution = list(sorted(institution))

field_of_study = set()
field_of_study.update(tuple(new_l4n2))
field_of_study = list(sorted(field_of_study))

In [15]:
num_paper = len(paper)
num_author = len(author)
num_institution = len(institution)
num_field_of_study = len(field_of_study)

nodeid_paper = list(range(num_paper))
nodeid_author = list(range(max(nodeid_paper)+1,max(nodeid_paper) + num_author + 1))
nodeid_institution = list(range(max(nodeid_author)+1,max(nodeid_author) + num_institution + 1))
nodeid_field_of_study = list(range(max(nodeid_institution)+1,max(nodeid_institution) + num_field_of_study + 1))
nodeid = nodeid_paper+nodeid_author+nodeid_institution+nodeid_field_of_study

nodename = paper+author+institution+field_of_study
nodetype = [0]*num_paper+[1]*num_author+[2]*num_institution+[3]*num_field_of_study

import pandas as pd
node = pd.DataFrame()
node['node_id']=nodeid
node['node_name']=nodename
node['node_type']=nodetype

new_feats_p = graph['node_feat_dict']['paper'][paper]
feats_p = []
for fs in new_feats_p:
    feats_p.append(','.join(list(map(str,fs))))

ss = ''
for si in range(len(feats_p[0])):
    if si == 0:
        ss += '0'
    else:
        ss += ',0'

feats_a = [ss]*num_author
feats_t = [ss]*num_institution
feats_c = [ss]*num_field_of_study


    
node['feats']=feats_p+feats_a+feats_t+feats_c

In [14]:
len(feats_p[0])

1202

### 重新编号link_id

In [16]:
node_a = node[node['node_type'] == 1][['node_id','node_name']].rename(columns={'node_name':'node_id1'})
new_link1_id1 = pd.merge(link1,node_a,how='left',on='node_id1')['node_id'].tolist()  
node_i = node[node['node_type'] == 2][['node_id','node_name']].rename(columns={'node_name':'node_id2'})
new_link1_id2 = pd.merge(link1,node_i,how='left',on='node_id2')['node_id'].tolist()  

new_link2_id1 = pd.merge(link2,node_a,how='left',on='node_id1')['node_id'].tolist()  
node_p = node[node['node_type'] == 0][['node_id','node_name']].rename(columns={'node_name':'node_id2'})
new_link2_id2 = pd.merge(link2,node_p,how='left',on='node_id2')['node_id'].tolist()  

new_link3_id2 = pd.merge(link3,node_p,how='left',on='node_id2')['node_id'].tolist()  
node_p = node[node['node_type'] == 0][['node_id','node_name']].rename(columns={'node_name':'node_id1'})
new_link3_id1 = pd.merge(link3,node_p,how='left',on='node_id1')['node_id'].tolist()  

new_link4_id1 = pd.merge(link4,node_p,how='left',on='node_id1')['node_id'].tolist()  
node_f = node[node['node_type'] == 3][['node_id','node_name']].rename(columns={'node_name':'node_id2'})
new_link4_id2 = pd.merge(link4,node_f,how='left',on='node_id2')['node_id'].tolist()  

link1['new_node_id1']=new_link1_id1
link1['new_node_id2']=new_link1_id2

link2['new_node_id1']=new_link2_id1
link2['new_node_id2']=new_link2_id2

link3['new_node_id1']=new_link3_id1
link3['new_node_id2']=new_link3_id2

link4['new_node_id1']=new_link4_id1
link4['new_node_id2']=new_link4_id2

links = pd.concat([link1,link2,link3,link4],axis=0)

### 重新编号label_id

In [17]:
import pandas as pd
labels = pd.DataFrame()
labels['node_name'] = label_idx[0]
labels['label'] = label['paper'][label_idx[0]]
node_p = node[node['node_type'] == 0]
labels = pd.merge(labels,node_p,how='left',on='node_name')

### 保存文件

In [20]:
file_name = 'ogb/node.dat'
with open(file_name, "w") as f:
        for nid, nname, ntype, fts in node.values:
            f.write(f"{int(nid)}\t{nname}\t{int(ntype)}\t{fts}\n")

In [21]:
file_name = 'ogb/link.dat'
with open(file_name, "w") as f:
            for old_nid1, old_nid2, ltype, fts, nid1, nid2 in links.values:
                f.write(f"{int(nid1)}\t{int(nid2)}\t{int(ltype)}\t{fts}\n")

In [23]:
l = np.array(range(len(labels)))
np.random.shuffle(l)
split = int(len(l)*0.7)
labels_test = labels.loc[l[:split]]
labels_train = labels.loc[l[split:]]

In [24]:
file_name = 'ogb/label.dat'
with open(file_name, "w") as f:
            for nname, label, nid, ntype, feats in labels_train.values:
                f.write(f"{int(nid)}\t{nname}\t{int(ntype)}\t{label}\n")
                
file_name = 'ogb/label.dat.test'
with open(file_name, "w") as f:
            for nname, label, nid, ntype, feats in labels_test.values:
                f.write(f"{int(nid)}\t{nname}\t{int(ntype)}\t{label}\n")

In [55]:
graph

{'edge_index_dict': {('author',
   'affiliated_with',
   'institution'): array([[      0,       1,       2, ..., 1134645, 1134647, 1134648],
         [    845,     996,    3197, ...,    5189,    4668,    4668]]),
  ('author',
   'writes',
   'paper'): array([[      0,       0,       0, ..., 1134647, 1134648, 1134648],
         [  19703,  289285,  311768, ...,  657395,  671118,  719594]]),
  ('paper',
   'cites',
   'paper'): array([[     0,      0,      0, ..., 736388, 736388, 736388],
         [    88,  27449, 121051, ..., 421711, 427339, 439864]]),
  ('paper',
   'has_topic',
   'field_of_study'): array([[     0,      0,      0, ..., 736388, 736388, 736388],
         [   145,   2215,   3205, ...,  21458,  22283,  31934]])},
 'edge_feat_dict': None,
 'node_feat_dict': {'paper': array([[-0.095379,  0.040758, -0.210948, ...,  0.061569, -0.027663,
          -0.133832],
         [-0.151047, -0.107315, -0.221964, ...,  0.345754, -0.027737,
          -0.218527],
         [-0.114799, -0.1759

In [53]:
from ogb import nodeproppred #import DglNodePropPredDataset

dataset = nodeproppred.NodePropPredDataset(name = 'ogbn-arxiv')

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph, label = dataset[0] # graph: dgl graph object, label: torch tensor of shape (num_nodes, num_tasks)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:09<00:00,  8.36it/s]


Extracting dataset/arxiv.zip
Loading necessary files...
This might take a while.


100%|██████████| 1/1 [00:00<00:00, 9822.73it/s]

Processing graphs...
Saving...


In [54]:
graph

{'edge_index': array([[104447,  15858, 107156, ...,  45118,  45118,  45118],
        [ 13091,  47283,  69161, ..., 162473, 162537,  72717]]),
 'edge_feat': None,
 'node_feat': array([[-0.057943, -0.05253 , -0.072603, ...,  0.173364, -0.172796,
         -0.140059],
        [-0.1245  , -0.070665, -0.325202, ...,  0.068524, -0.372111,
         -0.301036],
        [-0.080242, -0.023328, -0.183787, ...,  0.109919,  0.117589,
         -0.139883],
        ...,
        [-0.22053 , -0.036568, -0.402199, ...,  0.11336 , -0.161393,
         -0.145171],
        [-0.138236,  0.040885, -0.251811, ..., -0.08929 , -0.041253,
         -0.376132],
        [-0.029875,  0.268417, -0.161124, ...,  0.120807,  0.077647,
         -0.091018]], dtype=float32),
 'node_year': array([[2013],
        [2015],
        [2014],
        ...,
        [2020],
        [2020],
        [2020]]),
 'num_nodes': 169343}